In [1]:
import numpy as np
import numpy as np
import pandas as pd
import WindFarmGenetic # wind farm layout optimization using genetic algorithms classes
from datetime import datetime
import os
from sklearn.svm import SVR
import pickle


In [2]:
import os
os.getcwd()

'/home/naveen/arena/hackathon/shell/hackathon/python/Ver0_N'

In [3]:
##
wt_N=50  #number of turbines
NA_loc=[]  #index of obstructed locations
population_size=100  #population size/number of layouts for genetic algorithms
n_mc_samples=10000 #population/layout for SVM training
iteration_times=100  #number of iterations

# wind farm size, cells
cols_cells = 10  # number of cells each row
rows_cells = 10  # number of cells each column
cell_width = 400 # unit (TBD)--> have to be computed.

#genetic algorithmic parameters.
elite_rate = 0.2 
cross_rate = 0.6
mutate_rate = 0.1

random_rate = 0.5
num_runs=100 #number of independent runs with random initialization.


In [4]:
data_folder = "naveen_data" #temporary code related files. (MOdify it as per your specification)
svr_model_filename = 'svr_1s1d_N_13.svr' #svr training file.
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

wds_data_folder = "{}/wds".format(data_folder)
if not os.path.exists(wds_data_folder):
    os.makedirs(wds_data_folder)
    
# initial populations saved folder
init_pops_data_folder = "{}/init_data".format(data_folder)
if not os.path.exists(init_pops_data_folder):
    os.makedirs(init_pops_data_folder)

#result folder
results_data_folder = data_folder+"/results"
if not os.path.exists(results_data_folder):
    os.makedirs(results_data_folder)
    
#result folder for sugga genetic algorithm
sg_result_folder = "{}/sg".format(results_data_folder)
if not os.path.exists(sg_result_folder):
    os.makedirs(sg_result_folder)

In [5]:
import WindFarmGenetic
wfg = WindFarmGenetic.WindFarmGenetic(rows=rows_cells, cols=cols_cells, N=wt_N, NA_loc=NA_loc, pop_size=population_size,
                                      iteration=iteration_times,cell_width=cell_width, elite_rate=elite_rate,
                                             cross_rate=cross_rate, random_rate=random_rate, mutate_rate=mutate_rate)

In [6]:
for i in range(num_runs):
    wfg.gen_init_pop_NA()
    wfg.save_init_pop_NA("{}/init_{}.dat".format(init_pops_data_folder, i),
                         "{}/init_{}_NA.dat".format(init_pops_data_folder, i))

In [7]:
##Generate initial layouts: single layout--> 1d array of size row*col, layouts are set of num_training_sample_SVM different(random) layouts.  
train_mc_layouts, train_mc_layouts_NA = WindFarmGenetic.LayoutGridMCGenerator.gen_mc_grid_with_NA_loc(rows_cells,
                                                                                                      cols_cells,
                                                                                                      n_mc_samples,
                                                                                                      wt_N, NA_loc,
                                                                                                      "{}/mc_layout.dat".format(wds_data_folder),
                                                                                                      "{}/mc_layout_NA.dat".format(wds_data_folder))

layouts = np.genfromtxt("{}/mc_layout.dat".format(wds_data_folder), delimiter="  ", dtype=np.int32)

In [8]:
import WindFarmGenetic

wfg.mc_gen_xy_NA_hackathon(rows=rows_cells, cols=cols_cells, layouts=layouts, n=n_mc_samples, N=wt_N,  xfname="{}/x.dat".format(wds_data_folder),
                 yfname="{}/y.dat".format(wds_data_folder))

layouts: 100%|██████████| 10000/10000 [1:17:04<00:00,  2.16it/s]


In [6]:
x_original = pd.read_csv("{}/x.dat".format(wds_data_folder), header=None, nrows=rows_cells * cols_cells,
                         delim_whitespace=True, dtype=np.float32)
x_original = x_original.values

In [7]:
y_original = pd.read_csv("{}/y.dat".format(wds_data_folder), header=None, nrows=rows_cells * cols_cells,
                         delim_whitespace=True, dtype=np.float32)
y_original = y_original.values.flatten()

In [8]:
print(x_original.shape)
print(y_original.shape)

(100, 2)
(100,)


In [9]:
# create a SVR object and specify the kernal and other parameters
svr_model = SVR(kernel='rbf', C=2000.0, gamma=0.3, epsilon=.1)
# build the SVR power distribution model
svr_model.fit(x_original, y_original)

# save the SVR model to a file
pickle.dump(svr_model, open("{}/{}".format(wds_data_folder, svr_model_filename), 'wb'))

In [10]:
y_original

array([1.262066, 1.239304, 1.23055 , 1.229595, 1.228972, 1.231479,
       1.237167, 1.246812, 1.269636, 1.3298  , 1.231762, 1.205606,
       1.194989, 1.191217, 1.192288, 1.195313, 1.201483, 1.214201,
       1.240566, 1.309875, 1.22168 , 1.194623, 1.182284, 1.178334,
       1.178772, 1.182646, 1.18987 , 1.203885, 1.233446, 1.304872,
       1.217071, 1.190021, 1.177864, 1.173529, 1.174499, 1.176944,
       1.184476, 1.199587, 1.229349, 1.302895, 1.21692 , 1.189931,
       1.17777 , 1.173242, 1.17544 , 1.178668, 1.184909, 1.198483,
       1.229098, 1.302954, 1.219139, 1.191898, 1.180821, 1.17819 ,
       1.179483, 1.182453, 1.189618, 1.202172, 1.230747, 1.304921,
       1.222815, 1.197593, 1.187364, 1.184341, 1.185109, 1.189412,
       1.195355, 1.20807 , 1.235648, 1.308609, 1.234811, 1.211244,
       1.202014, 1.199757, 1.200277, 1.204203, 1.21014 , 1.222301,
       1.248417, 1.314861, 1.25715 , 1.237941, 1.232836, 1.230476,
       1.23136 , 1.233241, 1.238194, 1.248763, 1.268416, 1.328

In [11]:
del x_original
del y_original

In [12]:
##### SUGGA: support vector regression guided genetic algorithm
result_arr = np.zeros((num_runs, 2), dtype=np.float32)
for i in range(0, num_runs):  # run times
    print("run number: {} ...".format(i))
    wfg.load_init_pop_NA("{}/init_{}.dat".format(init_pops_data_folder, i),
                         "{}/init_{}_NA.dat".format(init_pops_data_folder, i))
    run_time, eta = wfg.sugga_genetic_alg(i, svr_model=svr_model, result_folder=sg_result_folder)
    result_arr[i, 0] = run_time
    result_arr[i, 1] = eta
time_stamp = datetime.now().strftime("%Y%m%d%H%M%S")
filename = "{}/result_sugga_{}.dat".format(sg_result_folder, time_stamp)
np.savetxt(filename, result_arr, fmt='%f', delimiter="  ")

layouts:   0%|          | 0/100 [00:00<?, ?it/s]

run number: 0 ...
Support vector regression guided genetic algorithm starts....
generation 0...


layouts:   0%|          | 0/100 [00:00<?, ?it/s]

[498.88763 496.26953 495.67276 502.00125 503.6059  498.46503 503.04343
 496.86047 496.52985 499.28455 501.78824 498.11252 495.34186 501.04242
 503.68616 497.78333 497.94165 498.34088 493.42215 492.0402  495.82742
 496.56116 503.81915 497.24997 496.75323 497.18637 498.1196  501.46463
 498.743   503.03635 501.16495 497.9488  492.53537 493.99237 499.291
 501.10867 498.37964 499.50137 498.63556 499.0552  499.84842 494.1147
 496.55942 500.46683 500.52954 502.7972  496.667   504.57523 499.83267
 502.36874 498.4432  496.31015 495.44818 491.42178 498.13644 498.0107
 500.5001  504.59195 498.87943 495.68674 499.10962 493.2068  497.81262
 499.2539  496.68542 500.71735 495.25732 491.7898  498.94183 493.03433
 499.25302 498.8226  491.85025 497.51096 495.85153 499.67526 496.6654
 502.00067 497.84885 498.0973  500.0395  491.705   501.94916 503.939
 496.1593  494.73495 498.80365 494.7972  495.78156 498.059   492.84747
 495.02835 497.0916  498.19016 496.56912 499.3705  500.9726  499.31705
 499.19937 49

layouts:  57%|█████▋    | 57/100 [00:11<00:08,  4.80it/s]

KeyboardInterrupt: 

### Shell Power Computation Package

In [1]:
import sys 
sys.path.append('../Shell_Code_Modified/')

In [2]:
import Farm_Evaluator_Vec

In [3]:
total_energy, turbine_power=Farm_Evaluator_Vec.get_AEP_results()

In [4]:
total_energy

array([1.2523587, 1.1595447, 1.1450489, 1.0768385, 1.176319 , 1.1840583,
       1.13568  , 1.079648 , 1.0935222, 1.1273198, 1.0859367, 1.167106 ,
       1.1016386, 1.124391 , 1.1225449, 1.100678 , 1.1647563, 1.2282459,
       1.1581823, 1.197542 , 1.2453799, 1.2442212, 1.2059841, 1.1259376,
       1.2380294, 1.1106503, 1.1325871, 1.1829029, 1.1153044, 1.1439447,
       1.1912581, 1.1264657, 1.1022447, 1.1789318, 1.1049944, 1.2037905,
       1.220672 , 1.1373823, 1.2131999, 1.1142861, 1.199062 , 1.141587 ,
       1.1335138, 1.1493688, 1.0722657, 1.0965657, 1.2305766, 1.2662541,
       1.1084578, 1.0826805], dtype=float32)

In [5]:
turbine_power

505.4506365966797